In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 5.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=7046e3533ee457fd78e8cfe7b233997ec1bec4b9aba069abf2c1b147720755f3
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=3c34df641f471b314fbacc9d8e57b736fa429a18a116e422a52d4760abbf6ebe
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [3]:
fashion_mist = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels) = fashion_mist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.5052 - accuracy: 0.8204 - val_loss: 0.4303 - val_accuracy: 0.8478
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3754 - accuracy: 0.8603 - val_loss: 0.3617 - val_accuracy: 0.8558
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3462 - accuracy: 0.8696 - val_loss: 0.3712 - val_accuracy: 0.8605


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4078 - accuracy: 0.8534 - val_loss: 0.3023 - val_accuracy: 0.8908
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2664 - accuracy: 0.9019 - val_loss: 0.2792 - val_accuracy: 0.9012
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2144 - accuracy: 0.9211 - val_loss: 0.2633 - val_accuracy: 0.9023


Epoch 1/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.4067 - accuracy: 0.8515 - val_loss: 0.2993 - val_accuracy: 0.8865
Epoch 2/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.2649 - accuracy: 0.9022 - val_loss: 0.2573 - val_accuracy: 0.9032
Epoch 3/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.2102 - accuracy: 0.9209 - val_loss: 0.2572 - val_accuracy: 0.9047


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3856 - accuracy: 0.8604 - val_loss: 0.2991 - val_accuracy: 0.8885
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2401 - accuracy: 0.9122 - val_loss: 0.2526 - val_accuracy: 0.9088
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1792 - accuracy: 0.9326 - val_loss: 0.2538 - val_accuracy: 0.9122


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5902 - accuracy: 0.7874 - val_loss: 0.4826 - val_accuracy: 0.8138
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4508 - accuracy: 0.8323 - val_loss: 0.4309 - val_accuracy: 0.8408
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4231 - accuracy: 0.8419 - val_loss: 0.4604 - val_accuracy: 0.8348


INFO:tensorflow:Oracle triggered exit


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        55344     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2211920   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,269,354
Trainable params: 2,269,354
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1283 - accuracy: 0.9523 - val_loss: 0.2685 - val_accuracy: 0.9110
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0908 - accuracy: 0.9671 - val_loss: 0.2938 - val_accuracy: 0.9052
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0609 - accuracy: 0.9779 - val_loss: 0.3680 - val_accuracy: 0.9110
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0449 - accuracy: 0.9834 - val_loss: 0.3959 - val_accuracy: 0.9127
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0327 - accuracy: 0.9881 - val_loss: 0.4471 - val_accuracy: 0.9083
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0278 - accuracy: 0.9901 - val_loss: 0.5491 - val_accuracy: 0.9048
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0222 - accuracy: 0.9921 - val_loss: 0.5791 - val_accuracy